### 0. To save

In [169]:
#df.to_csv('FINAL_PREPROCESSED.csv', index=False)

In [170]:
#df.to_csv('/home/jovyan/__RHI/FINAL_PREPROCESSED.csv', index=False)

### 1. Libraries

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/home/jovyan/__RHI/full_dataset.csv', index_col=0)

In [3]:
df.head()

,Unnamed: 0,url,title,location,type,level,salaryRange,salary,company,minSalary,maxSalary,currency,salaryPeriod,position,datePosted,validThrough,jobDescription,skills
0,0,https://ai-jobs.net/job/32218-data-science-con...,View full details of `Data Science Content Int...,Remote,Internship,Entry-level,USD 9K - 11K,NaN,NannyML,9000.0,11000.0,USD,YEAR,Data Science Content Intern (Remote),2022-11-09 22:55:46,2022-12-24 00:00:00,Description NannyML - creators of an Open Sour...,"['Bayesian', 'Content creation', 'GitHub', 'Ma..."
1,1,https://ai-jobs.net/job/32029-python-php-senio...,View full details of `Python / PHP Senior Deve...,"Remote, EU",Full Time,Senior-level,EUR 65K - 80K,NaN,Beatopia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['APIs', 'GCP', 'Google Cloud', 'Machine Learn..."
2,2,https://ai-jobs.net/job/30723-data-scientist-r...,View full details of `Data Scientist - Researc...,"Remote, Hybrid Available (Chicago)",Full Time,Senior-level,USD 55K - 120K,NaN,"HFR, Inc.",55000.0,120000.0,USD,YEAR,"Data Scientist - Research, Development &amp; C...",2022-10-26 20:46:21,2022-12-10 00:00:00,"Company Description: HFR, Inc. (HFR) was fo...","['Computer Science', 'Data Analytics', 'Data v..."
3,3,https://ai-jobs.net/job/30166-postdoctoral-res...,View full details of `Postdoctoral Research As...,"Oak Ridge, TN, United States",Temporary,Mid-level,NaN,NaN,Oak Ridge National Laboratory,NaN,NaN,NaN,NaN,Postdoctoral Research Associate - Boundary Pla...,2022-10-21 20:53:22,2022-12-05 00:00:00,Overview: Oak Ridge National Laboratory is th...,"['CAD', 'Engineering', 'Matlab', 'Physics', 'P..."
4,4,https://ai-jobs.net/job/30091-data-science-and...,View full details of `Data Science and Machine...,Remote- USA,Full Time,Senior-level,USD 150K - 200K,NaN,Libertas Funding,150000.0,200000.0,USD,YEAR,Data Science and Machine Learning Manager,2022-10-21 02:15:41,2022-12-05 00:00:00,USA Remote Position POSITION SUMMARY: The Dat...,"['Agile', 'Airflow', 'APIs', 'AWS', 'CI/CD', '..."


### 2. Level (grade), Salary

In [4]:
df.level.unique()

array(['Entry-level', 'Senior-level', 'Mid-level', nan, 'Executive-level'],
      dtype=object)

In [5]:
def striping_grade(grade):
    if not pd.isna(grade):
        return grade.replace('-level', '')
    else:
        return grade

In [6]:
df['level'] = df['level'].apply(striping_grade)

In [7]:
def get_currency(sr, mins, curr):
    if (not pd.isna(sr)) and pd.isna(curr):
        return str(sr)[:3]
    else:
        return curr

In [8]:
df['currency'] = df.apply(lambda row : get_currency(row['salaryRange'], row['minSalary'], row['currency']), axis = 1)

In [9]:
def get_min_salary(sr, mins):
    if (not pd.isna(sr)) and pd.isna(mins):
        
        return float(str(sr)[4:str(sr).find('K')]) * 1000
    else:
        return mins
    
def get_max_salary(sr, maxs):
    if (not pd.isna(sr)) and pd.isna(maxs):
        s = str(sr)[str(sr).find('-') + 2:]
        return float(s[:s.find('K')]) * 1000
    else:
        return maxs

In [10]:
df['minSalary'] = df.apply(lambda row : get_min_salary(row['salaryRange'], row['minSalary']), axis = 1)
df['maxSalary'] = df.apply(lambda row : get_max_salary(row['salaryRange'], row['maxSalary']), axis = 1)

In [11]:
def convert_to_usd(curr, s):
    eur_usd_rate = 1.02
    gbp_usd_rate = 1.18
    
    if curr=='EUR':
        return s * eur_usd_rate
    elif curr=='GBP':
        return s * gbp_usd_rate
    else:
        return s

In [12]:
def change_curr_to_usd(curr):
    if (not pd.isna(curr)) and (curr!='USD'):
        return 'USD'
    else:
        return curr

In [13]:
df['minSalary'] = df.apply(lambda row : convert_to_usd(row['currency'], row['minSalary']), axis = 1)
df['maxSalary'] = df.apply(lambda row : convert_to_usd(row['currency'], row['maxSalary']), axis = 1)

In [14]:
df['currency'] = df['currency'].apply(change_curr_to_usd)

### 3. Skills binary encoding

In [16]:
df.skills = df['skills'].apply(eval)

In [17]:
df = df.join(pd.crosstab((s:=df['skills'].explode()).index, s))

In [18]:
df.head()

,Unnamed: 0,url,title,location,type,level,salaryRange,salary,company,minSalary,...,UX,UX Research,Unstructured data,VR,Visual SLAM,Weka,XGBoost,XML,ggplot2,spaCy
0,0,https://ai-jobs.net/job/32218-data-science-con...,View full details of `Data Science Content Int...,Remote,Internship,Entry,USD 9K - 11K,NaN,NannyML,9000.0,...,0,0,0,0,0,0,0,0,0,0
1,1,https://ai-jobs.net/job/32029-python-php-senio...,View full details of `Python / PHP Senior Deve...,"Remote, EU",Full Time,Senior,EUR 65K - 80K,NaN,Beatopia,66300.0,...,0,0,0,0,0,0,0,0,0,0
2,2,https://ai-jobs.net/job/30723-data-scientist-r...,View full details of `Data Scientist - Researc...,"Remote, Hybrid Available (Chicago)",Full Time,Senior,USD 55K - 120K,NaN,"HFR, Inc.",55000.0,...,0,0,0,0,0,0,0,0,0,0
3,3,https://ai-jobs.net/job/30166-postdoctoral-res...,View full details of `Postdoctoral Research As...,"Oak Ridge, TN, United States",Temporary,Mid,NaN,NaN,Oak Ridge National Laboratory,NaN,...,0,0,0,0,0,0,0,0,0,0
4,4,https://ai-jobs.net/job/30091-data-science-and...,View full details of `Data Science and Machine...,Remote- USA,Full Time,Senior,USD 150K - 200K,NaN,Libertas Funding,150000.0,...,0,0,0,0,0,0,1,0,0,0


### 4. Extracting salary fork from text

In [20]:
df['money'] = df.jobDescription.str.findall('(\$[0-9,.]+)')

In [216]:
money_indices = df[df.salaryRange.isna()&(df.max_money>5000)].index

In [22]:
import numpy as np

In [23]:
def fork(money, sr):
    salary_fork = np.NaN
    
    if pd.isna(sr):
        if isinstance(money, float):
            salary_fork = np.NaN
        elif money:
            if len(money) == 1:
                float_m = float(money[0][1:].replace(',', '').strip('.'))
                if float_m >=25000:
                    salary_fork = [float_m, float_m]
            elif len(money) == 2:
                float_ms = [float(money[0][1:].replace(',', '').strip('.')), float(money[1][1:].replace(',', '').strip('.'))]
                if float_ms[0] >= 25000 and float_ms[1] / float_ms[0] < 2:
                    salary_fork = float_ms
            else:
                salary_fork = np.NaN

    return salary_fork

In [24]:
df['salaryFork'] = df.apply(lambda row : fork(row['money'], row['salaryRange']), axis = 1)

In [25]:
df[(~df.salaryFork.isna())&df.salaryRange.isna()]

,Unnamed: 0,url,title,location,type,level,salaryRange,salary,company,minSalary,...,Unstructured data,VR,Visual SLAM,Weka,XGBoost,XML,ggplot2,spaCy,money,salaryFork
7,7,https://ai-jobs.net/job/29719-postdoctoral-res...,View full details of `POSTDOCTORAL RESEARCH AS...,"Beltsville, MD",Temporary,Mid,NaN,NaN,USDA-ARS Animal Biosciences and Biotechnology,NaN,...,0,0,0,0,0,0,0,0,"[$74,950]","[74950.0, 74950.0]"
145,145,https://ai-jobs.net/job/33123-applied-scientis...,View full details of `Applied Scientist - Text...,"Remote, US or Europe",Full Time,Senior,NaN,NaN,Veritone,NaN,...,0,0,0,0,0,0,0,0,"[$140,000.00.]","[140000.0, 140000.0]"
178,178,https://ai-jobs.net/job/33100-power-bi-developer/,View full details of `Power BI Developer` vacancy,"Dallas, TX, United States",Full Time,NaN,NaN,NaN,Logic20/20 Inc.,NaN,...,0,0,0,0,0,0,0,0,"[$90,000]","[90000.0, 90000.0]"
213,213,https://ai-jobs.net/job/33058-senior-manager-d...,"View full details of `Senior Manager, Data Eng...",Remote (US or Canada),Full Time,Senior,NaN,NaN,1Password,NaN,...,0,0,0,0,0,0,0,0,"[$200,000.00]","[200000.0, 200000.0]"
244,244,https://ai-jobs.net/job/33080-senior-manager-a...,"View full details of `Senior Manager, Analytic...",Remote (US or Canada),Full Time,Senior,NaN,NaN,1Password,NaN,...,0,0,0,0,0,0,0,0,"[$191,000.00]","[191000.0, 191000.0]"
433,433,https://ai-jobs.net/job/32808-seniorstaff-data...,View full details of `Senior/Staff Data Engine...,"Toronto, ON, Canada",Full Time,Senior,NaN,NaN,Wish,NaN,...,0,0,0,0,0,0,0,0,"[$168,000, $240,000]","[168000.0, 240000.0]"
520,520,https://ai-jobs.net/job/32731-senior-data-anal...,View full details of `Senior Data Analyst I` v...,United States,Full Time,Senior,NaN,NaN,Unite Us,NaN,...,0,0,0,0,0,0,0,0,"[$85,000]","[85000.0, 85000.0]"
524,524,https://ai-jobs.net/job/32691-data-science-dir...,View full details of `Data Science Director- M...,"Chicago, IL/Remote",Full Time,Executive,NaN,NaN,OppFi,NaN,...,0,0,0,0,0,0,0,0,"[$150,000.]","[150000.0, 150000.0]"
556,556,https://ai-jobs.net/job/32716-data-engineer-ii/,View full details of `Data Engineer II` vacancy,Remote,Full Time,Senior,NaN,NaN,RECUR,NaN,...,0,0,0,0,0,0,0,0,"[$110,000, $100]","[110000.0, 100.0]"
561,561,https://ai-jobs.net/job/32654-senior-software-...,View full details of `Senior Software Engineer...,Remote (United States),Full Time,Senior,NaN,NaN,Rec Room,NaN,...,0,0,0,0,0,0,0,0,"[$160,000]","[160000.0, 160000.0]"


In [26]:
def min_salary_from_fork(fork, minSalary, sr):
    if pd.isna(minSalary) and pd.isna(sr) and not isinstance(fork, float):
        ms = fork[0]
    else:
        ms = minSalary
    return ms

In [27]:
df['minSalary'] = df.apply(lambda row : min_salary_from_fork(row['salaryFork'], row['minSalary'], row['salaryRange']), axis = 1)

In [266]:
df[(~df.salaryFork.isna())&df.salaryRange.isna()]

,url,title,location,type,level,salaryRange,salary,company,minSalary,maxSalary,...,Unstructured data,VR,Visual SLAM,Weka,XGBoost,XML,ggplot2,spaCy,money,salaryFork
7,https://ai-jobs.net/job/29719-postdoctoral-res...,View full details of `POSTDOCTORAL RESEARCH AS...,"Beltsville, MD",Temporary,Mid-level,NaN,NaN,USDA-ARS Animal Biosciences and Biotechnology,74950.0,NaN,...,0,0,0,0,0,0,0,0,"[$74,950]","[74950.0, 74950.0]"
145,https://ai-jobs.net/job/33123-applied-scientis...,View full details of `Applied Scientist - Text...,"Remote, US or Europe",Full Time,Senior-level,NaN,NaN,Veritone,140000.0,NaN,...,0,0,0,0,0,0,0,0,"[$140,000.00.]","[140000.0, 140000.0]"
178,https://ai-jobs.net/job/33100-power-bi-developer/,View full details of `Power BI Developer` vacancy,"Dallas, TX, United States",Full Time,NaN,NaN,NaN,Logic20/20 Inc.,90000.0,NaN,...,0,0,0,0,0,0,0,0,"[$90,000]","[90000.0, 90000.0]"
213,https://ai-jobs.net/job/33058-senior-manager-d...,"View full details of `Senior Manager, Data Eng...",Remote (US or Canada),Full Time,Senior-level,NaN,NaN,1Password,200000.0,NaN,...,0,0,0,0,0,0,0,0,"[$200,000.00]","[200000.0, 200000.0]"
244,https://ai-jobs.net/job/33080-senior-manager-a...,"View full details of `Senior Manager, Analytic...",Remote (US or Canada),Full Time,Senior-level,NaN,NaN,1Password,191000.0,NaN,...,0,0,0,0,0,0,0,0,"[$191,000.00]","[191000.0, 191000.0]"
433,https://ai-jobs.net/job/32808-seniorstaff-data...,View full details of `Senior/Staff Data Engine...,"Toronto, ON, Canada",Full Time,Senior-level,NaN,NaN,Wish,168000.0,NaN,...,0,0,0,0,0,0,0,0,"[$168,000, $240,000]","[168000.0, 240000.0]"
520,https://ai-jobs.net/job/32731-senior-data-anal...,View full details of `Senior Data Analyst I` v...,United States,Full Time,Senior-level,NaN,NaN,Unite Us,85000.0,NaN,...,0,0,0,0,0,0,0,0,"[$85,000]","[85000.0, 85000.0]"
524,https://ai-jobs.net/job/32691-data-science-dir...,View full details of `Data Science Director- M...,"Chicago, IL/Remote",Full Time,Executive-level,NaN,NaN,OppFi,150000.0,NaN,...,0,0,0,0,0,0,0,0,"[$150,000.]","[150000.0, 150000.0]"
556,https://ai-jobs.net/job/32716-data-engineer-ii/,View full details of `Data Engineer II` vacancy,Remote,Full Time,Senior-level,NaN,NaN,RECUR,110000.0,NaN,...,0,0,0,0,0,0,0,0,"[$110,000, $100]","[110000.0, 100.0]"
561,https://ai-jobs.net/job/32654-senior-software-...,View full details of `Senior Software Engineer...,Remote (United States),Full Time,Senior-level,NaN,NaN,Rec Room,160000.0,NaN,...,0,0,0,0,0,0,0,0,"[$160,000]","[160000.0, 160000.0]"


In [28]:
def max_salary_from_fork(fork, maxSalary, sr):
    if pd.isna(maxSalary) and not isinstance(fork, float) and pd.isna(sr):
        if fork[0]==fork[1]:
            ms = maxSalary
        else:
            ms = fork[1]
    else:
        ms = maxSalary
    return ms

In [29]:
df['maxSalary'] = df.apply(lambda row : max_salary_from_fork(row['salaryFork'], row['maxSalary'], row['salaryRange']), axis = 1)

In [31]:
df.drop(columns=['money', 'salaryFork'], inplace=True)

In [33]:
df.drop(columns=['salary'], inplace=True)

In [34]:
df.head()

,Unnamed: 0,url,title,location,type,level,salaryRange,company,minSalary,maxSalary,...,UX,UX Research,Unstructured data,VR,Visual SLAM,Weka,XGBoost,XML,ggplot2,spaCy
0,0,https://ai-jobs.net/job/32218-data-science-con...,View full details of `Data Science Content Int...,Remote,Internship,Entry,USD 9K - 11K,NannyML,9000.0,11000.0,...,0,0,0,0,0,0,0,0,0,0
1,1,https://ai-jobs.net/job/32029-python-php-senio...,View full details of `Python / PHP Senior Deve...,"Remote, EU",Full Time,Senior,EUR 65K - 80K,Beatopia,66300.0,81600.0,...,0,0,0,0,0,0,0,0,0,0
2,2,https://ai-jobs.net/job/30723-data-scientist-r...,View full details of `Data Scientist - Researc...,"Remote, Hybrid Available (Chicago)",Full Time,Senior,USD 55K - 120K,"HFR, Inc.",55000.0,120000.0,...,0,0,0,0,0,0,0,0,0,0
3,3,https://ai-jobs.net/job/30166-postdoctoral-res...,View full details of `Postdoctoral Research As...,"Oak Ridge, TN, United States",Temporary,Mid,NaN,Oak Ridge National Laboratory,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,4,https://ai-jobs.net/job/30091-data-science-and...,View full details of `Data Science and Machine...,Remote- USA,Full Time,Senior,USD 150K - 200K,Libertas Funding,150000.0,200000.0,...,0,0,0,0,0,0,1,0,0,0


In [36]:
#df.to_csv('FINAL_PREPROCESSED.csv', index=False)

### 5. Translation

In [41]:
!pip install pycld2 

  Using cached pycld2-0.41-cp39-cp39-linux_x86_64.whl


In [42]:
import pycld2 as cld2

In [43]:
import numpy as np

In [46]:
import regex

In [47]:
RE_BAD_CHARS = regex.compile(r"[\p{Cc}\p{Cs}]+")

def remove_bad_chars(text):
    return RE_BAD_CHARS.sub("", text)

def lang_detection(jd):
    if isinstance(jd, str):
        jd = remove_bad_chars(jd)
        return cld2.detect(jd, returnVectors=False)[2][0][1]
    else:
        return np.nan

In [48]:
df['lang'] = df.apply(lambda row : lang_detection(row['jobDescription']), axis = 1)

In [49]:
df.lang.unique()

array(['en', nan, 'fr', 'nl', 'de', 'es', 'pt', 'ja', 'sv', 'hu', 'ko',
       'da', 'pl', 'uk', 'it', 'cs'], dtype=object)

In [64]:
df['titleLang'] = df.apply(lambda row : lang_detection(row['title']), axis = 1)

In [65]:
df.titleLang.unique()

array(['en', 'de', 'fr', 'un', 'sv', 'pl', 'pt'], dtype=object)

In [50]:
import nltk

In [51]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/jovyan/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to

True

In [52]:
!pip install deep-translator

  Using cached deep_translator-1.9.1-py3-none-any.whl (30 kB)


In [53]:
from deep_translator import GoogleTranslator

In [55]:
def translating(text, orig_lang):
    if isinstance(text, str) and orig_lang!='en':
        x = nltk.tokenize.sent_tokenize(text)
        s = ''
        for sentence in x:
            # t = ts.google(sentence, from_language=orig_lang, to_language='en')
            if isinstance(sentence, str):
                t = GoogleTranslator(source=orig_lang, target='en').translate(sentence)
                if isinstance(t, str):
                    s = s + ' ' + str(t)
        return s
    else:
        return text

In [56]:
df['jobDescription'] = df.apply(lambda row: translating(row['jobDescription'], row['lang']), axis=1)

In [61]:
df[(df.lang!='en')&(~df.lang.isna())].jobDescription

10       Company description Created in 2002, Talan no...
11       Company descriptionA pioneer of Consulting 4....
18       Company DescriptionSia Partners is a next-gen...
24       Company descriptionA pioneer of Consulting 4....
26       Company descriptionA pioneer of Consulting 4....
                              ...                        
3633     CI&amp;amp;T is a digital specialist, partner...
3637     Data Engineer GCP M/F/D Who are we? Capco is ...
3670     The TeamWorking in the AutoVu division of Gen...
3688     You might not know Incognia (yet!) but we are...
3750     CI&amp;amp;T is a digital specialist, partner...
Name: jobDescription, Length: 264, dtype: object

In [73]:
def title_translating(text, orig_lang, title_orig_lang):
    if isinstance(text, str) and (title_orig_lang!='en') and (title_orig_lang==orig_lang):
            if isinstance(text, str):
                t = GoogleTranslator(source=orig_lang, target='en').translate(text)
                if isinstance(t, str):
                    return t
                else:
                    return text
            else:
                return text
    else:
        return text

In [74]:
df['title'] = df.apply(lambda row: title_translating(row['title'], row['lang'], row['titleLang']), axis=1)

### 6. Job Description Text Preprocessing

In [89]:
import string
import re
import html

In [119]:
RE_BAD_CHARS1 = regex.compile(r"[\p{Cc}\p{Cs}]+")
RE_BAD_CHARS2 = regex.compile(r"(x[0-9][0-9])")

punct = re.compile('[' + re.escape(string.punctuation) + ']')
digits = re.compile('[' + re.escape(string.digits) + ']')

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)


def clean (str_): 
    str_ = ''.join(e for e in str_ if e.isalnum() or e==' ')
    str_ = html.unescape(str_ )
    str_ = str_.lower()
    str_ = re.sub('\n',' ',str_)
    str_ = re.sub(punct,r' ', str_)
    str_ = re.sub(digits,r' ', str_)
    str_ = re.sub(r'\s+',r' ', str_)
    str_ = emoji_pattern.sub(r'', str_)
    str_ = str_.strip()
    
    return str_


def remove_bad_chars(text, company):
    if isinstance(text, str):
        s = text.replace("\n", " ")
        s = s.replace(company, " ")
        s = RE_BAD_CHARS1.sub("", s)
        s = clean(s)
        s = RE_BAD_CHARS2.sub("", s)
        s = s.replace(clean(company.lower()), " ")
        s = s.replace("  ", " ").replace("   ", " ")
        return s
    else:
        return text

In [120]:
df['cleanJobDescription'] = df.apply(lambda row: remove_bad_chars(row['jobDescription'], row['company']), axis=1)

In [123]:
df.loc[50]['cleanJobDescription'] 

'company description www com is a global technology and hpc leader that provides data products services and solutions in earth science data science sensing and monitoring our unique portfolio supports our clients in efficiently and responsibly solving complex digital energy transition natural resource environmental and infrastructure challenges for a more sustainable future employs around people worldwide and is listed on the euronext paris sa isin job description is looking for an experienced enthusiastic talented and highly motivated senior geoscience data scientist to join our geoscience rampampd teamour geoscience rampampd groupis a new and expanding team within geoscience consisting of talented geoscience data scientists senior research scientists ml engineers and software developers we work alongside multidisciplinary geoscientists to provide innovative solutions to our clients challenging problems within the energy sector by joining usyou will play a vital role in the continual 

#### Lemmatization

In [124]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [126]:
from nltk.stem import WordNetLemmatizer

In [129]:
lemmatizer = WordNetLemmatizer()

def lemmatize (str_):
    if isinstance(str_, str):
        words = str_.strip().split(' ')
        words = ' '.join([lemmatizer.lemmatize(word) for word in words ])

        return words
    else:
        return str_

In [132]:
df['lemmeJobDescription'] = df['cleanJobDescription'].apply(lemmatize)

In [133]:
df.loc[50]['lemmeJobDescription'] 

'company description www com is a global technology and hpc leader that provides data product service and solution in earth science data science sensing and monitoring our unique portfolio support our client in efficiently and responsibly solving complex digital energy transition natural resource environmental and infrastructure challenge for a more sustainable future employ around people worldwide and is listed on the euronext paris sa isin job description is looking for an experienced enthusiastic talented and highly motivated senior geoscience data scientist to join our geoscience rampampd teamour geoscience rampampd groupis a new and expanding team within geoscience consisting of talented geoscience data scientist senior research scientist ml engineer and software developer we work alongside multidisciplinary geoscientists to provide innovative solution to our client challenging problem within the energy sector by joining usyou will play a vital role in the continual development of

#### Stop words

In [134]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [135]:
from nltk.corpus import stopwords

In [144]:
nltk_stopwords = stopwords.words('english')

In [145]:
custom_sw = ['company', 'job', 'business', 'description', 'dresscode', 'compensation', 'benefit', 
                       'pension', 'equal', 'employment', 'employ', 'employer', 'opportunity', 'sexual', 'sex', 
                       'diversity', 'orientation', 'gender', 'national', 'identity', 'www', 'com', 
                       'meantal', 'health', 'discount', 'gym','dental', 'care',
                      'such', 'despite'] 

In [146]:
sw = nltk_stopwords + custom_sw

In [154]:
def remove_stop_words(str_, stopwords, custom_stopwords):
    if isinstance(str_, str):
        words = str_.strip().split(' ')
        cleaned_words = []

        for word in words:
            new_word = word
            for stopword in custom_stopwords:
                if stopword in new_word:
                    new_word = new_word.replace(stopword, '')
                    
            if new_word != '':
                cleaned_words.append(new_word)
        


        words = ' '.join([word for word in cleaned_words if word not in stopwords])

        return words
    else:
        return str_

In [155]:
df['finalJobDescription'] = df['lemmeJobDescription'].apply(
    lambda x: remove_stop_words(x, sw, custom_sw))


In [156]:
df.loc[50]['finalJobDescription'] 

'global technology hpc leader provides data product service solution earth science data science sensing monitoring unique portfolio support client efficiently responsibly solving plex digital energy transition natural resource environmental infrastructure challenge sustainable future around people worldwide listed euronext paris sa isin looking experienced enthusiastic talented highly motivated senior geoscience data scientist join geoscience rampampd teamour geoscience rampampd groupis new expanding team within geoscience consisting talented geoscience data scientist senior research scientist ml engineer software developer work alongside multidisciplinary geoscientists provide innovative solution client challenging problem within energy sector joining usyou play vital role continual development digital transformation tool geoscience analytic techniquesthe successful candidate responsible supporting development implementation new technology workflow extract transform load manipulate ex

### 7. Position and Details from Title

In [85]:
def clean_position(title):
    if not pd.isna(title):
        return title.replace('View full details of `', '').replace('` vacancy', '')
    else:
        return title

In [86]:
df['title'] = df['title'].apply(lambda x: clean_position(x))

In [87]:
def clear_brackets(title):
    if not pd.isna(title):
        s = re.sub("\[.*?\]","",title)
        s = re.sub("\(.*?\)","",s)
        return s
    else:
        return title

In [90]:
df['position'] = df['title'].apply(lambda x: clear_brackets(x))

In [91]:
def excluding_details(title):
    if not pd.isna(title):
        if " - " in title:
            s = title[title.find(" - "):]
        elif " for" in title:
            s = title[title.find(" for"):]
        else:
            s = np.nan
        return s
    else:
        return np.nan

In [92]:
df['details'] = df['position'].apply(lambda x: excluding_details(x))

In [93]:
def getting_position(title, excl):
    if not pd.isna(title):
        if not pd.isna(excl):
            s = title.replace(excl, '')
            return s
        else:
            return title
    else:
        return title

In [94]:
df['position'] = df.apply(lambda row: getting_position(row['position'], row['details']), axis=1)

In [95]:
def clearing_details(details):
    if not pd.isna(details):
        if details.startswith(' - '):
            return details.replace(' - ', '')
        else:
            return details.strip()
    else:
        return details

In [96]:
df['details'] = df['details'].apply(clearing_details)

In [100]:
df[['minSalary', 'maxSalary', 'jobDescription', 'position']].count()

minSalary         2022
maxSalary         1989
jobDescription    3719
position          3754
dtype: int64

### 8. New Scope Feature

In [162]:
def scoping(position):
    if 'manage' in str.lower(position) or 'direct' in str.lower(position):
        scope = 'management'
    elif 'data scien' in position.lower():
        scope = 'data science'
    elif 'machine learning enginee' in position.lower() or 'ml enginee' in position.lower():
        scope = 'ML engineering'
    elif 'data enginee' in position.lower():
        scope = 'data engineering'
    elif 'analy' in position.lower():
        scope = 'data analytics'
    elif 'software engineer' in position.lower() or 'software develope' in position.lower():
        scope = 'software engineering'
    elif 'research' in position.lower() or 'applied research' in position.lower() or 'applied scien' in position.lower():
        scope = 'research science'
    elif 'architect' in position.lower():
        scope = 'architecture'
    else:
        scope = 'other'
        
    return scope

In [163]:
df['scope'] = df['position'].apply(scoping)

### 9. Final Preprocessed Dataset

In [164]:
df.head()

,url,title,location,type,level,salaryRange,company,minSalary,maxSalary,currency,...,XML,ggplot2,spaCy,lang,titleLang,details,cleanJobDescription,lemmeJobDescription,finalJobDescription,scope
0,https://ai-jobs.net/job/32218-data-science-con...,Data Science Content Intern (Remote),Remote,Internship,Entry,USD 9K - 11K,NannyML,9000.0,11000.0,USD,...,0,0,0,en,en,NaN,description creators of an open source python ...,description creator of an open source python l...,creator open source python library looking mul...,data science
1,https://ai-jobs.net/job/32029-python-php-senio...,Python / PHP Senior Developer for Ambitious St...,"Remote, EU",Full Time,Senior,EUR 65K - 80K,Beatopia,66300.0,81600.0,USD,...,0,0,0,NaN,en,for Ambitious Startup,NaN,NaN,NaN,other
2,https://ai-jobs.net/job/30723-data-scientist-r...,"Data Scientist - Research, Development & Const...","Remote, Hybrid Available (Chicago)",Full Time,Senior,USD 55K - 120K,"HFR, Inc.",55000.0,120000.0,USD,...,0,0,0,en,en,"Research, Development & Construction",company description hfr was founded in and spe...,company description hfr wa founded in and spec...,hfr wa founded specializes area hedge fundalte...,data science
3,https://ai-jobs.net/job/30166-postdoctoral-res...,Postdoctoral Research Associate - Boundary Pla...,"Oak Ridge, TN, United States",Temporary,Mid,NaN,Oak Ridge National Laboratory,NaN,NaN,NaN,...,0,0,0,en,en,Boundary Plasma Physicist,overview is the largest us department of energ...,overview is the largest u department of energy...,overview largest u department energy science e...,research science
4,https://ai-jobs.net/job/30091-data-science-and...,Data Science and Machine Learning Manager,Remote- USA,Full Time,Senior,USD 150K - 200K,Libertas Funding,150000.0,200000.0,USD,...,0,0,0,en,en,NaN,usa remote position position summary the data ...,usa remote position position summary the data ...,usa remote position position summary data lead...,management


#### Total rows

In [165]:
df.shape[0]

3754

#### Rows with salary (at least minSalary)

In [166]:
df[~df.minSalary.isna()].shape[0]

2022

#### Rows with Job Descritption

In [167]:
df[~df.jobDescription.isna()].shape[0]

3719

#### Rows with both Salary and Job Descritption

In [168]:
df[(~df.minSalary.isna())&(~df.jobDescription.isna())].shape[0]

2006